# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 37](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=37)*

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-09-25"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink

In [3]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "true"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [4]:
# Set default env to "usdf" and try before PUSH to repo.

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.usdf
)  # TODO try with "usdf" before push (else "summit")

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    limit=limit,
)
allrep = AllReports(allsrc=allsrc)

DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr=registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=500&min_day_obs=20240925&max_day_obs=20240926
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr=registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=500&min_day_obs=20240925&max_day_obs=20240926
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr=registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=500&min_day_obs=20240925&max_day_obs=20240926
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr=registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=500&min_day_obs=20240925&max_day_obs=20240926
DBG get_exposures qstr=registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=5

<head>
<style>
code {
  font-family: Consolas,"courier new";
  color: crimson;
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 105%;
}
</style>
</head>

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-09-25 to 2024-09-26 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac)
* [Summary plots of whole night](#Summary-plots-of-whole-night)
* [Time Accounting](#Time-Accounting)
* [Jira Tickets](#Jira-Tickets)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log)
* [Narrative Log](#Narrative-Log)
* [Developer Only Section](#Developer-Only-Section)

## Night Report 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```
Daily checkouts, started at ~10:30 CLT and went well.

UPS failed, it was not charging and ran out of battery a couple of times. We follow AuxTel Recovery after Shutdown procedure, missed one system so checkout failed, issue fixed and checkout passed.

We started venting at ~16:20CLT.

Calibrations skipped.

Observations started 20:37 CLT and finished 01:30 CLT. No issues to report.
```

Telescope Status: ATTCS and LATISS in STANDBY.
Dome and Telescope at park position.

*Authors: Bruno Quint, Jacqueline Seron, Paulina Venegas, Yijung Kang*

### Telescope: Simonyi

...

*Authors: Bruno Quint, Jacqueline Seron, Paulina Venegas, Yijung Kang*

```
Commercial power was back at 15:25. We started the day plans for BLOCK-T151 Simonyi Hexapods Startup around 18:33 pm UTC. Then we warmed up hexapods (BLOCK-T3 and -T4) without an issue. Right after warming up we put hexapods back DISABLED and ran the MTRotator startup block (~19:30 PM UTC). MTRotator start-up, M1M3 opening and closing exercise were proceeded before lunch without any critical issue. 

After dinner, we changed MTPtg configuration to proceed Soak Test w/ Rotator and mtmount-ccw-only test (BLOCK-T170). The Scheduler1 was cycled several times to get some changes from Tiago R. There was no overcurrent (>3.5A) occurring during the soak test. 

Around 1:15am, we stopped scheduler and changed the MTPtg setup to monitor both MTRotator and MTMount as requested on the testcase. Both hexapods and rotator were all move to the origin and set to Standby. PDUs were off (BLOCK-T192 and BLOCK-T162).
```

Telescope Status: Simonyi telescope is parked at the Zenith. MTHexapod1, MTHexapod2, and MTRotator are on standby, and PDUs are off. MTPtg is set to monitor both MTRotator and MTMount. 

*Authors: Bruno Quint, Jacqueline Seron, Paulina Venegas, Yijung Kang*

-------------

## Almanac

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-09-25**

,(times in UTC),
Moon Rise,2024-09-25 06:22:17.988,
Moon Set,2024-09-25 16:08:11.923,
Moon Illumination,46%,(% illuminated)
Astronomical Twilight (morning),2024-09-25 09:07:53.478,(-18 degrees)
Astronomical Twilight (evening),2024-09-25 00:00:53.995,(-18 degrees)
Solar Midnight,2024-09-25 04:34:25.909,
Nautical Twilight (morning),2024-09-25 09:36:06.914,(-12 degrees)
Nautical Twilight (evening),2024-09-24 23:32:39.307,(-12 degrees)
Civil Twilight (morning),2024-09-25 10:04:03.933,(-6 degrees)
Civil Twilight (evening),2024-09-24 23:04:40.798,(-6 degrees)


## Summary plots of whole night 

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting

In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTCam
Total Night (HH:MM:SS),9:06:59
Total Exposure (HH:MM:SS),4:54:19
Number of exposures,1366
Number of slews,46
Total Detector Read (HH:MM:SS),0:52:21
Mean Detector Read (HH:MM:SS),0:00:02
Total Slew (HH:MM:SS),0:33:45
Mean Slew (HH:MM:SS),0:00:44
Total Idle (HH:MM:SS),3:38:54
Total fault loss (HH:MM:SS),0:00:00


## Jira Tickets 
(Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20240925&max_day_obs=20240926) in `confluence_url`.

## BLOCKS Observed
(TODO: BLOCKS observed) 

## Data Log
NOTE: *modified all_sources.py:uniform_field_counts()*
NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

### Experimental Exposure field Tally ENABLED

Tally exposures with specific values of selected fields

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 0 total exposures**

...

,flat,bias,dark
0,829,433,104


observation_reason

,flat,dark,sflat hi,bias
0,1239,104,3,20


science_program

,E749,E1086,E1076,E1075
0,1339,3,12,12


## TODO According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument. (that tally has been added below)
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [14]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

-------------

Note: *Added counts of Exposures with messages*

In [15]:
for instrum, recs in allsrc.exp_src.messages.items():
    md(f'**{instrum} {len(recs)} Exposures with messages**')
    if len(recs) > 0:
        df = allsrc.tally_exposure_flags(instrum)
        display(df.style.hide(axis="index"))

## Narrative Log

In [16]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-09-25 05:31:10.509012** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>sending shutdown script</pre>


- **2024-09-25 05:19:19.979973** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>adding scheduler/stop.py to the bottom of the list</pre>


- **2024-09-25 03:43:49.131964** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We send ATPtg to enable and continue.</pre>


- **2024-09-25 03:39:46.975210** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>ATPtg went to fault, azimuth out of range
Salindex 200890</pre>


- **2024-09-25 03:20:40.057576** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We solve the issue and continue</pre>


...

- **2024-09-25 14:21:13.074630** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Cannot transition ATMCS - The script complains saying that cannot get summary state from it. Sal index 201021</pre>


- **2024-09-25 13:50:14.599266** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Checkout finished successfully</pre>


- **2024-09-25 13:28:48.964191** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['AuxTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>AuxTel Checkouts starting</pre>


- **2024-09-25 13:15:03.690457** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Day crew arriving in the control room - Bruno Quint and Jacqueline Seron</pre>


-------------

-----------
------------


# Developer Only Section 
Contains stuff only expected to be useful to developers.

May also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [17]:
print(dev_msg)
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False

Imported lsst.ts.logging_and_reporting from local packages.


In [18]:
# Display overview of Report context
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
"""
)

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-09-25 12:00:00**.
- min_dayobs='2024-09-25', max_dayobs='2024-09-26'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [19]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [20]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [4]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20240925&max_day_obs=20240926

None
- Got 4 records.  


### Overview for Service: `exposurelog` [0]

...

None
- name 'verbose' is not defined 


### Overview for Service: `narrativelog` [80]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-09-25T00%3A00%3A00&max_date_added=2024-09-26T00%3A00%3A00

None
- Got 80 records.  


In [21]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [22]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-10-31 13:11:21.403200
